In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on April 28, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-04-28'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  15475   
1                         10,508 (67.9%)   
2                          4,967 (32.1%)   
3        Among those with race specified   
4                              78 (1.6%)   
5                             232 (4.7%)   
6                          1,223 (24.6%)   
7                              12 (0.2%)   
8                          2,527 (50.9%)   
9                            895 (18.0%)   
10                                   NaN   
11                        10,461 (67.6%)   
12                         5,014 (32.4%)   
13  Among those with ethnicity specified   
14                         1,957 (39.0%)   
15                         3,057 (61.0%)   

                                          \
                                   18-44   
0                                 267400   
1                        168,876 (63.2%)   
2                         98,524 (36.8%)   
3        Among those with race specified   
4                             907 (0.9%)   
5                           5,323 (5.4%)   
6                         27,541 (28.0%)   
7                             377 (0.4%)   
8                         49,200 (49.9%)   
9                         15,176 (15.4%)   
10                                   NaN   
11                       176,607 (66.0%)   
12                        90,793 (34.0%)   
13  Among those with ethnicity specified   
14                        27,849 (30.7%)   
15                        62,944 (69.3%)   

                                          \
                                   45-64   
0                                 266612   
1                        156,837 (58.8%)   
2                        109,775 (41.2%)   
3        Among those with race specified   
4                             792 (0.7%)   
5                           5,863 (5.3%)   
6                         34,783 (31.7%)   
7                             305 (0.3%)   
8                         53,811 (49.0%)   
9                         14,221 (13.0%)   
10                                   NaN   
11                       168,500 (63.2%)   
12                        98,112 (36.8%)   
13  Among those with ethnicity specified   
14                        24,305 (24.8%)   
15                        73,807 (75.2%)   

                                          \
                                   65-74   
0                                  82788   
1                         40,291 (48.7%)   
2                         42,497 (51.3%)   
3        Among those with race specified   
4                             234 (0.6%)   
5                           1,961 (4.6%)   
6                         13,727 (32.3%)   
7                              98 (0.2%)   
8                         22,387 (52.7%)   
9                           4,090 (9.6%)   
10                                   NaN   
11                        46,562 (56.2%)   
12                        36,226 (43.8%)   
13  Among those with ethnicity specified   
14                         6,369 (17.6%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)